In [16]:
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

In [17]:
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'ml-metadata.sqlite3'
connection_config.sqlite.connection_mode = 3
store = metadata_store.MetadataStore(connection_config)

In [18]:
data_type = metadata_store_pb2.ArtifactType()
data_type.name = 'Dataset'
data_type.properties['day'] = metadata_store_pb2.INT
data_type.properties['split'] = metadata_store_pb2.STRING
data_type_id = store.put_artifact_type(data_type)

In [19]:
model_type = metadata_store_pb2.ArtifactType()
model_type.name = 'SavedModel'
model_type.properties['version'] = metadata_store_pb2.INT
model_type.properties['name'] = metadata_store_pb2.STRING
model_type_id = store.put_artifact_type(model_type)

In [20]:
trainer_type = metadata_store_pb2.ExecutionType()
trainer_type.name = 'Trainer'
trainer_type.properties['state'] = metadata_store_pb2.STRING
trainer_type_id = store.put_execution_type(trainer_type)

In [21]:
data_artifact = metadata_store_pb2.Artifact()
data_artifact.uri = 'dataset.zip'
data_artifact.properties['day'].int_value = 1
data_artifact.properties['split'].string_value = 'train'
data_artifact.type_id = data_type_id
data_artifact_id = store.put_artifacts([data_artifact])

In [22]:
trainer_run = metadata_store_pb2.Execution()
trainer_run.type_id = trainer_type_id
trainer_run.properties['state'].string_value = 'RUNNING'
run_id = store.put_executions([trainer_run])

In [27]:
input_event = metadata_store_pb2.Event()
input_event.artifact_id = data_artifact_id[0]
input_event.execution_id = run_id[0]
input_event.type = metadata_store_pb2.Event.DECLARED_INPUT
store.put_events([input_event])

In [28]:
model_artifact = metadata_store_pb2.Artifact()
model_artifact.uri = 'model.pb2'
model_artifact.properties['version'].int_value = 1
model_artifact.properties['name'].string_value = 'MNIST-v1'
model_artifact.type_id = model_type_id
model_artifact_id = store.put_artifacts([model_artifact])

In [29]:
output_event = metadata_store_pb2.Event()
output_event.artifact_id = model_artifact_id[0]
output_event.execution_id = run_id[0]
output_event.type = metadata_store_pb2.Event.DECLARED_OUTPUT
store.put_events([output_event])

In [30]:
experiment_type = metadata_store_pb2.ContextType()
experiment_type.name = 'Experiment'
experiment_type.properties['note'] = metadata_store_pb2.STRING
experiment_type_id = store.put_context_type(experiment_type)

In [31]:
my_experiment = metadata_store_pb2.Context()
my_experiment.type_id = experiment_type_id
my_experiment.name = 'exp1'
my_experiment.properties['note'].string_value = 'My first experiment.'
experiment_id = store.put_contexts([my_experiment])

In [32]:
attribution = metadata_store_pb2.Attribution()
attribution.artifact_id = model_artifact_id[0]
attribution.context_id = experiment_id[0]

association = metadata_store_pb2.Association()
association.execution_id = run_id[0]
association.context_id = experiment_id[0]

store.put_attributions_and_associations([attribution], [association])

In [34]:
store.get_artifacts()

[id: 1
 type_id: 1
 uri: "dataset.zip"
 properties {
   key: "day"
   value {
     int_value: 1
   }
 }
 properties {
   key: "split"
   value {
     string_value: "train"
   }
 }, id: 2
 type_id: 2
 uri: "model.pb2"
 properties {
   key: "name"
   value {
     string_value: "MNIST-v1"
   }
 }
 properties {
   key: "version"
   value {
     int_value: 1
   }
 }]

In [35]:
store.get_artifacts_by_context(experiment_id[0])

[id: 2
 type_id: 2
 uri: "model.pb2"
 properties {
   key: "name"
   value {
     string_value: "MNIST-v1"
   }
 }
 properties {
   key: "version"
   value {
     int_value: 1
   }
 }]

In [36]:
store.get_artifacts_by_type('Dataset')

[id: 1
 type_id: 1
 uri: "dataset.zip"
 properties {
   key: "day"
   value {
     int_value: 1
   }
 }
 properties {
   key: "split"
   value {
     string_value: "train"
   }
 }]